In [6]:
import requests
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
import numpy as np




In [3]:
 def get_token():
        with open(".token") as t:
            tok = t.readline()
        return tok.strip()

In [10]:
def temp_prediction(days_in_future: int, lat: float, long: float):
    # fetch data 
    dataset = 'prismc-tmax-daily'
    my_token = get_token() #'REDACTED'

    my_url = 'https://api.dclimate.net/apiv3/grid-history/' + dataset + '/' + str(lat) + '_' + str(long)
    head = {"Authorization": my_token}
    
    print("Downloading...")
    r = requests.get(my_url, headers=head)
    data = r.json()["data"]
    
    print("Preprocessing...")
    index = pd.to_datetime(list(data.keys()))
    values = [float(s.split()[0]) if s else None for s in data.values()]
    series = pd.Series(values, index=index)
    df = series.to_frame(name='Value')
    df = df[~df.index.astype(str).str.contains('02-29')]

    # algorithm 
    print("Fitting the model")
    hw_model = ExponentialSmoothing(df["Value"],
                                    trend="add",
                                    seasonal="add",
                                    seasonal_periods=365,
                                    damped=False).fit() #use_boxcox='log'

    hw_fitted = hw_model.fittedvalues

    hw_resid = hw_model.resid

    # Adding the mean of the residuals to correct the bias.
    print("Forecasting...")
    py_hw = hw_model.forecast(days_in_future) + np.mean(hw_resid)

    # output
    return (py_hw.values[-1])



In [3]:
nb_days = 10
results_exponential = {}
results_linear = {}

In [4]:
towns = {
    "Miami": [25.762329613614614, -80.19114735100034],
    "New York": [40.64136425563865, -73.78201731266307],
    "Las Vegas": [36.169819817986365, -115.14034125787191]
}


In [13]:
# prediction form exponential
for town in towns:
    expo = temp_prediction(nb_days, towns[town][0], towns[town][1])
    results_exponential[str(town)] = (expo - 32) * 5/9
    
results_exponential

Downloading...
Preprocessing...
Fitting the model


/home/bm7/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: the 'damped' keyword is deprecated, use 'damped_trend' instead
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:595: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Forecasting...


/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:393: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


Downloading...
Preprocessing...
Fitting the model


...

Downloading...
Preprocessing...
Fitting the model


/home/bm7/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: the 'damped' keyword is deprecated, use 'damped_trend' instead
/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:595: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


Forecasting...


/home/bm7/.local/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:393: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


{'Miami': 25.119499873879853,
 'New York': 4.249870858046876,
 'Las Vegas': 13.920167110588771}

In [1]:
from Vacation_model import Dataset, Vacation

In [8]:
for town in towns:
    vacation = Vacation()
    d = Dataset()
    print("downloading data.....")
    d = d.download(town, "temp")
    print("preprocessing.....")
    ts = vacation.get_timeseries(d, 2001, 2022, "02-09", "02-10", 0, 24)
    years = vacation.calculate_yearly_value(ts,  metric="avg")
    y = np.array(list(years.values()))
    X = np.array(list(years.keys())).reshape(-1, 1)
    print("training.....")
    model = vacation.train(X, y)
    result = model.predict([[2022]])
    print(result)
    results_linear[str(town)] = int(result)
    result = 0
    
results_linear

downloading data.....
preprocessing.....
training.....
[74.92428428]
downloading data.....
preprocessing.....
training.....
[74.92428428]
downloading data.....
preprocessing.....
training.....
[74.92428428]


{'Miami': 74, 'New York': 74, 'Las Vegas': 74}